News before 2003 are crawled from "老资料网" (https://www.laoziliao.net/rmrb), where each line consists of one full news article starting with "第N版()专栏：", which I removed during preprocessing. News after 2004 are crawled from "人民日报图文数据库" (http://paper.people.com.cn/rmrb), each file consists of an article without the column, so they are treated seperately.

In [28]:
import os
import re
import pkuseg

stopfile = 'D:/人民日报/stopwords.txt'
rawdir = 'D:/人民日报/raw/'
yearlydir = 'D:/人民日报/1-year/'
frequencydir = 'D:/人民日报/frequency/'

with open(stopfile, 'r', encoding='utf-8') as sf:
    stopword_list = [word.strip('\n') for word in sf.readlines()]

seg = pkuseg.pkuseg()

In [ ]:
for i in range(1946, 2004):

    repr = str(i)
    print("Processing: " + repr + "...")

    rootdir = rawdir + repr + '年'
    newfile = yearlydir + repr + '.txt'
    countfile = frequencydir + repr + '.txt'

    paths = []
    counts = {}

    for root, dirs, files in os.walk(rootdir):
        for file in files:
            paths.append(os.path.join(root, file).encode('utf-8'))

    f = open(newfile,'wb')
    cf = open(countfile, 'wb')

    for i in paths:
        
        fileTrainRead = []
        with open(i, encoding='utf-8') as fileTrainRaw:
            for line in fileTrainRaw:
                fileTrainRead.append(re.sub(r"第.*版\(.*\)专栏：", "", line))

        # Each line a piece of news
        fileTrainSeg = []
        for news in fileTrainRead:
            temp = ""
            for word in seg.cut(news):
                if not (re.search('[^\.\-0-9a-zA-Z\u4e00-\u9fa5]+', word) or re.match('[\.\-\d]+', word)):
                    if word not in stopword_list:
                        temp += word + " "
                        if word in counts.keys():
                            counts[word] = counts[word] + 1
                        else:
                            counts[word] = 1
            temp = temp.rstrip()
            if temp:
                fileTrainSeg.append(temp)

        for i in range(len(fileTrainSeg)):
            f.write(fileTrainSeg[i].encode('utf-8'))
            f.write("\n".encode('utf-8'))
    
    counts_list = list(counts.items())
    counts_list.sort(key=lambda x:x[1], reverse=True)

    for (word, freq) in counts_list:
        word_freq = word + ' ' + str(freq) + '\n'
        cf.write(word_freq.encode('utf-8'))

    f.close()
    cf.close()

In [ ]:
for i in range(2004, 2024):
    
    repr = str(i)
    print("Processing: " + repr + "...")

    rootdir = rawdir + repr + '年'
    newfile = yearlydir + repr + '.txt'
    countfile = frequencydir + repr + '.txt'
    
    paths = []
    counts = {}

    for root, dirs, files in os.walk(rootdir):
        for file in files:
            paths.append(os.path.join(root, file).encode('utf-8'))

    f = open(newfile,'wb')
    cf = open(countfile, 'wb')
    
    for i in paths:

        fileTrainRead = []
        with open(i, encoding='utf-8') as fileTrainRaw:
            for line in fileTrainRaw:
                fileTrainRead.append(line)

        # Each file a piece of news
        fileTrainSeg = ""
        for sentence in fileTrainRead:
            temp = ""
            for word in seg.cut(sentence):
                if not (re.search('[^\.\-0-9a-zA-Z\u4e00-\u9fa5]+', word) or re.match('[\.\-\d]+', word)):
                    if word not in stopword_list:
                        temp += word + " "
                        if word in counts.keys():
                            counts[word] = counts[word] + 1
                        else:
                            counts[word] = 1
            if temp:
                fileTrainSeg += temp
        
        fileTrainSeg = fileTrainSeg.rstrip()
        f.write(fileTrainSeg.encode('utf-8'))
        f.write("\n".encode('utf-8'))

    counts_list = list(counts.items())
    counts_list.sort(key=lambda x:x[1], reverse=True)

    for (word, freq) in counts_list:
        word_freq = word + ' ' + str(freq) + '\n'
        cf.write(word_freq.encode('utf-8'))

    f.close()
    cf.close()